In [26]:
import dash
from jupyter_dash import JupyterDash
from dash import Dash
from dash import dcc
from dash import html
from dash import dash_table
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate

import plotly.express as px
import datetime as dt
import time
import numpy as np
import psycopg2

In [32]:
hostname = 'localhost'
database = 'rbike'
username = 'postgres'
pwd = 'admin'
port_id = 5432
conn= None
cur= None

try:
    conn = psycopg2.connect(
        host=hostname,
        dbname=database,
        user=username,
        password=pwd,
        port=port_id)

    cur = conn.cursor()

    insert_script= 'INSERT INTO customer (first_name, last_name, sex, email) VALUES (%s,%s,%s,%s)'
    insert_values= ('Mitchell', 'Pritchett','male', 'MVP@modern_family.com')
    cur.execute(insert_script,insert_values)
    
    conn.commit()

except Exception as error:
    print(error)

finally:
    if cur is not None:
        cur.close()
        conn.close()
    

In [2]:
tab_style = {
    'borderBottom': '1px solid #d6d6d6',
    'padding': '6px',
    'font-family':'Lucida Grande',
    'font-size': '15px',
    'fontWeight': 'bold'
}

tab_selected_style = {
    'borderTop': '1px solid #d6d6d6',
    'borderBottom': '1px solid #d6d6d6',
    'backgroundColor': '#000091',
    'color': 'white',
    'padding': '6px',
    'font-family':'Lucida Grande',
    'font-size': '15px'
}

header = {
    "font-family": "Lucida Grande",
    "font-size": "25px",
    "letter-spacing": "0.4px",
    "word-spacing": "2px",
    "color": "#000091",
    "font-weight": "700",
    "text-decoration": "none",
    "font-style": "normal",
    "font-variant": "small-caps",
    "text-transform": "none",
}

searchBox = {
    'width': '99.3%',
    'height': '200px',
    'margin-top':'10px',
    'background-color':'#dedede',
    'borderWidth': '1px',
    'borderStyle': 'solid',
    'borderRadius': '5px',
    'borderColor': '#dedede',
    'padding': '5px',
    'font-family':'Lucida Grande'
}


In [24]:
app = JupyterDash(__name__)
app.layout = html.Div(children=[
    html.H1(children=['RBike Database Management System'], style=header),
    html.Div(id='home', className='control_tabs',children=[
        dcc.Tabs(id='view_tabs',value='customer', children=[
            dcc.Tab(
                label='Bikes',
                value='bikes',
                style=tab_style,
                selected_style= tab_selected_style,
                children=[
                    
                ]
            ),
            dcc.Tab(
                label='Customers',
                value='customer',
                style=tab_style,
                selected_style= tab_selected_style,
                children=[
                    html.Div(style={'margin-top':'10px'},children=[
                        dash_table.DataTable(
                            id='customer_table',
                            columns=[
                                {'name':'first_name','id':'first_name','deletable':False,'renamable':False},
                                {'name':'last_name','id':'last_name','deletable':False,'renamable':False},
                                {'name':'sex','id':'sex','deletable':False,'renamable':False},
                                {'name':'email','id':'email','deletable':False,'renamable':False}],
                            data=[
                                {'first_name':'Lucas','last_name':'Lanz','sex':'male','email':'Lucas.lanz@gmail.com'},
                                {'first_name':'Matze','last_name':'Fischer','sex':'male','email':'Matze.fischer@yahoo.com'},
                                {'first_name':'Lias','last_name':'Musster','sex':'male','email':'Liasccdv@gmail.com'},
                                {'first_name':'Laura','last_name':'Engel','sex':'female','email':'Laurii@yas.de'},
                                {'first_name':'Miriam','last_name':'Teufel','sex':'female','email':'Miriam.teufel@web.de'},
                                {'first_name':'Luisa','last_name':'Menual','sex':'female','email':'lulu@web.de'},
                                {'first_name':'Jasmina','last_name':'Pascanovic','sex':'female','email':'jasminapascanovic@atos.net'}
                            ]
                        )
                    ])
                ]
            ),
            dcc.Tab(
                label='Bookings',
                value='bookings',
                style=tab_style,
                selected_style=tab_selected_style,
                children= [
                    html.Div(style= searchBox,children=[
                        html.H5(children=['Search Box'],style={'margin-top':'-2px'}),
                        dcc.Input(placeholder='what are you looking for?', type='text',id='input 1'),
                        dcc.Input(placeholder='what are you looking for?', type='text',id='input 2'),
                        html.P(children='hello',style={'margin-left':'90%'})
                    ])
                ]
            ),
            dcc.Tab(
                label='H.R.',
                value='hr',
                style=tab_style,
                selected_style=tab_selected_style,
                children= [
                    html.H1('Test')
                ]
            )
        ])
    ])
])

In [25]:
if __name__ == '__main__':
    app.run_server(debug=True, mode='external', host='127.0.0.1',port=8000)

Dash app running on http://127.0.0.1:8000/
